In [1]:
using Pkg
Pkg.activate("./demo/Project.toml");

  Activating project at `~/Documents/Jupyter Notebooks/DIY/demo`


In [2]:
using Random

In [3]:
function RandomSpin()
    """Creates a random unit vector"""
    θ = rand(0:1e-6:π)
    ϕ = rand(0:1e-6:2π)
    return [sin(θ)*cos(ϕ), sin(θ)*sin(ϕ), cos(θ)]
end

function AcceptanceProbability(Spin1, Spin2, n, βJ)
    """Computes the probability of adding Spin2 - neighbour of
    Spin1 - to the cluster. It uses the energy βJ and the vector 
    of reflection n"""
    return 1 - exp(min(0,-2βJ*transpose(Spin1)*n*transpose(Spin2)*n))
end

function ReflectSpin(S,n,state)
    """Reflectes the spin state[S] around the plane defined by the
    normal vector n"""
    state[S] -=  2(transpose(state[S])*n)*n
end

function GetNeighbours(x,L,L2)
    """Gets the coordinates of the 4 neighbour spins of the one in index x. It 
    uses periodic boundery conditions in a 2D lattice of length L (area L2). The 
    matriz it returns contains the neighbours as folows: Up-Left-Right-Down"""
    return [(x-1-L+L2)%L2+1,
            L*((x-1)÷L)+(x+L-2)%L+1,
            L*((x-1)÷L)+(x)%L+1,
            (x-1+L)%L2+1]
end

function Grow_Reflect(S, Cluster, n, βJ, L, L2, state)
    """Checks every neighbour of spin state[S] and adds them to the Cluster given
    a certain probability (See to AcceptanceProbability). Finally, it reflects
    the spin around the normal vector n (Refair to ReflectSpin)"""
    #Checks neighbours
    for Sn in GetNeighbours(S,L,L2)
        if Sn ∉ Cluster && rand(0:1e-15:1) < AcceptanceProbability(state[S],state[Sn],n,βJ)
            #Adds neighbour to Cluster
            push!(Cluster,Sn)
            #Checks neighbours of neighbour
            Grow_Reflect(Sn, Cluster, n, βJ, L, L2, state)
        end
    end
    #Reflects the spin
    ReflectSpin(S,n, state)
end

function NewState(state, L, L2, βJ)
    """Chooses a randon spin from the state with lattice sice L, and also a
    random normal vector. From there it builds the cluster using Wolff algorithm
    (See Grow_Reflect)"""
    n = RandomSpin()     #Initial random Normal Vector
    #n = [0,0,1]         #Just for testing
    S0 = rand(1:1:L2)    #Initial random Spin
    Cluster = [S0]       #Stores the indexes of the spins added to the cluster
    Grow_Reflect(S0, Cluster, n, βJ, L, L2, state) #Builds cluster and flips states for new state
    #print("\n",Cluster) #Just for testing
end

NewState (generic function with 1 method)

In [4]:
function RandomIsing()
    """Creates spins only in the z-axis (testing only)"""
    return [0, 0, rand(-1:2:1)]
end

RandomIsing (generic function with 1 method)

In [5]:
βJ = 10  #Energy of interaction (Quantified)
L = 3    #Lattice size
L2 = L*L #Lattice Area
#state = [RandomIsing() for i in 1:L2] #Just for testing
state = [RandomSpin() for i in 1:L2]   #The 2D lattice is store as a 1D-Array
#print(state)
NewState(state, L, L2, βJ) #New state
#print("\n",state)

[[0.28894226552948327, 0.9537027201702589, -0.08344752081720931], [-0.2580894480480774, -0.02644427471119372, 0.9657590471443888], [-0.7335488492784864, -0.5592568967026471, -0.38618364700325297], [0.15586974935896325, -0.6670663274333186, -0.7285102168394062], [-0.32814698992351093, -0.8639480498841066, 0.3819860208248312], [-0.8274220654082688, -0.3048321515586658, -0.47164614389564263], [0.5198231899716833, 0.4724667987243956, 0.7117295661069482], [0.02773861599470513, 0.23474529044683562, 0.9716610611708844], [0.35087239409370236, 0.2467269714785802, 0.9033351341600533]]
[[0.28894226552948327, 0.9537027201702589, -0.08344752081720931], [-0.2580894480480774, -0.02644427471119372, 0.9657590471443888], [-0.49226882317517523, -0.23425186480835025, 0.8383301673944095], [0.46667068163368586, -0.24841639383511765, 0.8488272911361339], [-0.32814698992351093, -0.8639480498841066, 0.3819860208248312], [-0.573199404179484, 0.03760665473330832, 0.818552492249614], [0.5198231899716833, 0.472466